# COURSE: Master Python for scientific programming by solving projects
## PROJECT: Time series filtering
#### TEACHER: Mike X Cohen, sincxpress.com
##### COURSE URL: udemy.com/course/maspy_x/?couponCode=202201

In [ ]:
# import all necessary modules
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import scipy.fftpack

# Generate a signal with random noise

In [ ]:
srate = 1000
time  = np.arange(0,10*srate)/srate
npnts = len(time)

data = np.random.randn(npnts)*5

# add 50 Hz "line noise"
linenoise = np.sin(2*np.pi*50*time)
data += linenoise

# spectral stuff
hz = np.linspace(0,srate/2,int(npnts/2)+1)


In [ ]:
def plotSignal(data):
  fig,ax = plt.subplots(1,2,figsize=(20,6))
  ax[0].plot(time,data)
  ax[0].set_xlabel('Time (s)')
  ax[0].set_title('Time domain')

  dataX = scipy.fftpack.fft(data/npnts)
  ax[1].plot(hz,np.abs(dataX[:len(hz)]))
  ax[1].set_xlim([0,150])
  ax[1].set_xlabel('Frequency (Hz)')
  ax[1].set_title('Frequency domain')


plotSignal(data)

# Notch out line noise

In [ ]:
f0 = 50  # Frequency to be removed from signal (Hz)
Q = 50  # Quality factor
# Design notch filter
b,a = signal.iirnotch(f0,Q,srate)

# evaluate the filter kernel
freq,h = signal.freqz(b,a,fs=srate)
plt.plot(freq,abs(h)**2)
plt.title("Frequency Response")
plt.ylabel("Amplitude (gain)")
plt.xlabel('Frequency (Hz)')
plt.xlim([0, 100])
plt.show()

In [ ]:
# apply the filter and inspect
notched = signal.filtfilt(b,a,data)

plotSignal(data)
plotSignal(notched)

In [ ]:
## create a function to plot two signals on top of each other

def plot2Signals(data1,data2):
  # create a figure
  fig,ax = plt.subplots(1,2,figsize=(15,4))

  # plot the time-domain
  ax[0].plot(time,data1,label='Original')
  ax[0].plot(time,data2,label='Filtered')
  ax[0].set_xlabel('Time (s)')
  ax[0].legend()
  ax[0].set_xlim([1,1.2])
  ax[0].set_title('Time domain')

  # plot the frequency-domain signal
  data1X = np.abs( scipy.fftpack.fft(data1/npnts) )
  data2X = np.abs( scipy.fftpack.fft(data2/npnts) )
  hz = np.linspace(0,srate/2,int(npnts/2)+1)
  ax[1].plot(hz,data1X[:len(hz)],label='Original')
  ax[1].plot(hz,data2X[:len(hz)],label='Filtered')
  ax[1].set_xlim([0,150])
  ax[1].legend()
  ax[1].set_xlabel('Frequency (Hz)')
  ax[1].set_title('Frequency domain')


In [ ]:
# now run that plotting function
plot2Signals(data,notched)

# High-pass FIR filter

In [ ]:
lowedge = 20 # Hz
fkern = signal.firwin(lowedge*10+1,lowedge/(srate/2),pass_zero=False)

# evaluate the filter kernel
freq,h = signal.freqz(fkern,1,fs=srate)
plt.plot(freq,abs(h)**2)
plt.title("Frequency Response")
plt.ylabel("Amplitude (gain)")
plt.xlabel('Frequency (Hz)')
plt.xlim([0, 100])
plt.show()

In [ ]:
# apply the filter and plot
highpass = signal.filtfilt(fkern,1,notched)

plotSignal(data)
plotSignal(highpass)
plot2Signals(data,highpass)

# Low-pass IIR filter

In [ ]:
lowcutoff = 30

# create filter coefficients
b,a = signal.butter(6,lowcutoff/(srate/2),btype='lowpass')

# evaluate the filter kernel
freq,h = signal.freqz(b,a,fs=srate)
plt.plot(freq,abs(h)**2)
plt.title("Frequency Response")
plt.ylabel("Amplitude (gain)")
plt.xlabel('Frequency (Hz)')
plt.xlim([0,100])
plt.show()



In [ ]:
bandpass = signal.filtfilt(b,a,highpass)

plotSignal(data)
plotSignal(bandpass)
plot2Signals(data,bandpass)

# Bonus: make a desert landscape

In [ ]:
filtdata = np.zeros((10,npnts))
centfreqs = np.linspace(6,15,10)

r = 0
for i in centfreqs:
  fwin = np.array([i-2,i+2])
  b = signal.firwin(2001,fwin/(srate/2),pass_zero=False)
  filtdata[r,:] = signal.filtfilt(b,1,data)
  r += 1


In [ ]:
plt.imshow(filtdata,aspect='auto',origin='lower',
           extent=[time[0],time[-1],centfreqs[0],centfreqs[-1]],cmap='Wistia')
plt.xlim([2,3])
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.title('Wistia color map')
plt.show()